## Creating Annotation Model for Gameplay

### Goal:
To develop a machine learning model that accurately identifies and annotates specific events and objects within gameplay videos, leveraging manually annotated frames to train the model.

### Purpose: 
- Automate Annotation: Reduce the manual effort required in annotating large volumes of gameplay data by automating the annotation process.
- Improve Model Accuracy: Utilize manually annotated frames to train a high-accuracy model, ensuring precise and reliable annotations in the gameplay videos.
- Facilitate Advanced Analytics: Enable deeper analysis and insights from the annotated gameplay data, supporting tasks such as event detection, player behavior analysis, and game strategy optimization.

### Tools:

<b>Data Preparation:</b>
pyautogui (for capturing periodic screenshots during gameplay),
opencv for extracting frames from gameplay videos

<b>Annotation Tools:</b>
Roboflow

<b>Frameworks:</b>
PyTorch,
TensorFlow

<b>Pre-trained Models:</b>
YOLO (You Only Look Once),
Faster R-CNN,

<b>Development and Testing:</b>
Visual Studio Code,
Jupyter Notebook,
GitHub or GitLab for version control

<b>Deployment:</b>
AWS,
Docker for containerization

In [23]:
from roboflow import Roboflow
import os
import dotenv
import torch
from ultralytics import YOLO
import cv2
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import torch

device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
dotenv.load_dotenv()
print(f"Using device: {device}")


Using device: mps


### Importing Manually Annotated Gameplay Frames

!pip install roboflow

In [33]:
rf = Roboflow(api_key= os.getenv("ROBOFLOW_APIKEY"))
project = rf.workspace("scotts-workspace").project("pokemon_platinum_ai")
version = project.version(15)
dataset = version.download("yolov9")

dataset_path = '/Users/scottpitcher/Desktop/python/Github/PokemonPlatinum.AI/pokemon_platinum_ai-16/data.yaml'

num_workers= os.cpu_count()

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to pokemon_platinum_ai-15 in yolov9:: 100%|██████████| 1842/1842 [00:00<00:00, 8297.37it/s]


Fatal Python error: PyEval_RestoreThread: the function must be called with the GIL held, after Python initialization and before Python finalization, but the GIL is released (the current Python thread state is NULL)
Python runtime state: initialized

Thread 0x000000037fb93000 (most recent call first):
  <no Python frame>

Thread 0x000000030b2ff000 (most recent call first):
  File "/opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 359 in wait
  File "/opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 655 in wait
  File "/Users/scottpitcher/Desktop/python/Github/PokemonPlatinum.AI/.venv/lib/python3.12/site-packages/tqdm/_monitor.py", line 60 in run
  File "/opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1073 in _bootstrap_inner
  File "/opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Pyth

: 

#### Initial Model Training (Broken into Three Parts Due to Computation Cost)

In [26]:
model = YOLO("yolov9s.pt")

model.train(
    data = dataset_path,
    epochs = 60,
    patience =12,
    imgsz= 640,
    device='cpu',
    batch= 8,
    amp = True,
    workers = num_workers,
    val = True,
    plots = True,
    name = "firstRun",
    exist_ok = True,
)
print("Starting TensorBoard...")
os.system(f"tensorboard --logdir models/annotation_runs/")

New https://pypi.org/project/ultralytics/8.2.87 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.78 🚀 Python-3.12.4 torch-2.3.1 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov9s.pt, data=/Users/scottpitcher/Desktop/python/Github/PokemonPlatinum.AI/pokemon_platinum_ai-16/data.yaml, epochs=60, time=None, patience=12, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=firstRun, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retin

train: Scanning /Users/scottpitcher/Desktop/python/Github/PokemonPlatinum.AI/pokemon_platinum_ai-16/train/labels... 2342 images, 96 backgrounds, 0 corrupt: 100%|██████████| 2342/2342 [00:00<00:00, 3135.64it/s]

train: WARNING ⚠️ /Users/scottpitcher/Desktop/python/Github/PokemonPlatinum.AI/pokemon_platinum_ai-16/train/images/2024-06-20_16-50-42_png.rf.1d6fc3ae8a5652b610a24f8a87a668f2.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /Users/scottpitcher/Desktop/python/Github/PokemonPlatinum.AI/pokemon_platinum_ai-16/train/images/2024-06-20_16-50-42_png.rf.3afc27919bc76e699492f028046e6b81.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /Users/scottpitcher/Desktop/python/Github/PokemonPlatinum.AI/pokemon_platinum_ai-16/train/images/2024-06-20_16-50-42_png.rf.aeb3e477fe447220212f4bd9c057e6d0.jpg: 1 duplicate labels removed
train: New cache created: /Users/scottpitcher/Desktop/python/Github/PokemonPlatinum.AI/pokemon_platinum_ai-16/train/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 60, len(boxes) = 14015. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-


val: Scanning /Users/scottpitcher/Desktop/python/Github/PokemonPlatinum.AI/pokemon_platinum_ai-16/valid/labels... 88 images, 9 backgrounds, 0 corrupt: 100%|██████████| 88/88 [00:00<00:00, 3522.67it/s]

val: New cache created: /Users/scottpitcher/Desktop/python/Github/PokemonPlatinum.AI/pokemon_platinum_ai-16/valid/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 6, len(boxes) = 507. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

Ultralytics YOLOv8.2.78 🚀 Python-3.12.4 torch-2.3.1 CPU (Apple M2)
YOLOv9s summary (fused): 486 layers, 7,215,076 parameters, 0 gradients, 27.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:53<00:00,  8.84s/it]


                   all         88        507      0.571      0.639      0.705      0.555
                   Bag          3          3      0.739          1      0.913      0.776
           FaintSymbol          1          1          1          0      0.995      0.597
               Pokemon          1          1          1          0      0.995      0.995
                   Run          3          3          1      0.909      0.995      0.913
            attackUsed          7          7      0.916          1      0.995      0.955
              bagItems          1          1      0.592          1      0.995      0.796
               bagMenu          1          1      0.414          1      0.995      0.995
             bagOption          1          1          1          0          0          0
             bagPocket          1          1      0.284          1      0.995      0.995
                 barry          2          2      0.721          1      0.995      0.796
                battl

sh: tensorboard: command not found


32512

In [ ]:
model = YOLO("runs/detect/firstRun/weights/best.pt")

model.train(
    data = dataset_path,
    epochs = 120,
    patience =2,
    imgsz= 640,
    device=device,
    batch= 8,
    amp = True,
    workers = num_workers,
    val = True,
    plots = True,
    # resume = True, # Start training off firstRun weights
    name = "secondRun",
    exist_ok = True,
)
print("Starting TensorBoard...")
os.system(f"tensorboard --logdir models/annotation_runs/")

### Final Training Loop

In [ ]:
model = YOLO("runs/detect/finalRun2/weights/best.pt")

model.train(
    data = dataset_path,
    epochs = 60,
    patience = 12,
    imgsz= 640,
    device=device,
    batch= 8,
    amp = True,
    workers = num_workers,
    val = True,
    plots = True,
    # resume = True, # Start training off secondRun weights
    name = "finalRun",
)
print("Starting TensorBoard...")
os.system(f"tensorboard --logdir models/annotation_runs/")

In [ ]:
import os

label_path = '/Users/scottpitcher/Desktop/python/Github/PokemonPlatinum.AI/route_203_images/train/labels/'
num_classes = 124

for label_file in os.listdir(label_path):
    with open(os.path.join(label_path, label_file), 'r') as f:
        lines = f.readlines()

    corrected_lines = []
    for line in lines:
        class_id = int(line.split()[0])
        if class_id < num_classes:
            corrected_lines.append(line)
        else:
            print(f"Ignoring corrupt label in {label_file}: {line.strip()}")

    with open(os.path.join(label_path, label_file), 'w') as f:
        f.writelines(corrected_lines)

print("Finished checking and correcting label files.")


In [ ]:
model = YOLO("runs/detect/firstRun/weights/best.pt")

In [ ]:
import os

label_path = '/Users/scottpitcher/Desktop/python/Github/PokemonPlatinum.AI/route_203_images/train/labels/'
num_classes = 124
class_counts = {i: 0 for i in range(num_classes)}

for label_file in os.listdir(label_path):
    with open(os.path.join(label_path, label_file), 'r') as f:
        lines = f.readlines()
        for line in lines:
            class_id = int(line.split()[0])
            if class_id < num_classes:
                class_counts[class_id] += 1

# Print class counts
for class_id, count in class_counts.items():
    print(f"Class {class_id}: {count} instances")


In [ ]:
model = torch.load('runs/detect/firstRun/weights/best.pt')

version.deploy(model_type='yolov9',model_path ='runs/detect/firstRun')

### Testing the model on a random image

In [30]:
best_path = "runs/detect/firstRun/weights/best.pt"
image_path = "/Users/scottpitcher/Desktop/python/Github/PokemonPlatinum.AI/annotated_images/images/2024-09-01_21-47-44_png.rf.06c8ac396e7ca2b5b01beda98f45bdd7.jpg"
model = YOLO(best_path)
results = model.predict(image_path)
results

for result in results:
    if result.boxes is not None:
        for box in result.boxes:
            class_id = box.cls.item()           # Get the class ID
            confidence = box.conf.item()        # Get the confidence score
            class_name = model.names[class_id]  # Map the class ID to the class name
            print(f"Object: {class_name}, Confidence: {confidence:.2f}")


Python(96636) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



image 1/1 /Users/scottpitcher/Desktop/python/Github/PokemonPlatinum.AI/annotated_images/images/2024-09-01_21-47-44_png.rf.06c8ac396e7ca2b5b01beda98f45bdd7.jpg: 640x448 1 battle, 1 healthOpponent, 1 healthUser, 1 levelOpponent, 1 levelUser, 1 opponentBattle, 1 userBattle, 237.7ms
Speed: 8.7ms preprocess, 237.7ms inference, 17.6ms postprocess per image at shape (1, 3, 640, 448)
Object: battle, Confidence: 0.95
Object: opponentBattle, Confidence: 0.90
Object: userBattle, Confidence: 0.88
Object: levelOpponent, Confidence: 0.85
Object: levelUser, Confidence: 0.85
Object: healthUser, Confidence: 0.80
Object: healthOpponent, Confidence: 0.80


In [28]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

best_path = "runs/detect/firstRun/weights/best.pt"
image_path = "/Users/scottpitcher/Desktop/python/Github/PokemonPlatinum.AI/annotated_images/images/2024-09-01_21-41-30_png.rf.6e06b983956e5c1342fdb8fd6b681d72.jpg"

model = YOLO(best_path)
results = model.predict(image_path)
results

for result in results:
    # Count the number of detected objects
    num_objects = len(result.boxes) if result.boxes is not None else 0
    print(f"Number of objects detected: {num_objects}")

    # Render and show the image with detected objects
    result_image = result.plot()
    result_image_bgr = cv2.cvtColor(result_image, cv2.COLOR_RGB2BGR) # Convert color scale from RGB to BGR
    plt.imshow(result_image_bgr) 
    plt.title(f"Detected Objects: {num_objects}")
    plt.axis('off')
    plt.show()

Python(94102) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



image 1/1 /Users/scottpitcher/Desktop/python/Github/PokemonPlatinum.AI/annotated_images/images/2024-09-01_21-41-30_png.rf.6e06b983956e5c1342fdb8fd6b681d72.jpg: 640x448 1 barry, 2 fences, 1 route203, 1 stairs, 1 tallGrass, 3 treeAreas, 1 user, 384.6ms
Speed: 135.5ms preprocess, 384.6ms inference, 18.5ms postprocess per image at shape (1, 3, 640, 448)
Number of objects detected: 10


#### Error handling: missing ["model"] attr from annot model
This occured due to the training being interrupted <br>
**Solution:** Grab the attr from another model with the same architecture, and reassign it

In [ ]:
model = torch.load("runs/detect/firstRun/weights/best.pt")
print(model['model'])

In [ ]:
yolo_model = torch.load('yolov9s.pt')
trained_model =  torch.load("runs/detect/tunedModel3/weights/best.pt")

same_keys = yolo_model['model'].state_dict().keys() == trained_model['model'].state_dict().keys()
print(f"Same architecture: {same_keys}")

same_weights = all(torch.equal(yolo_model['model'].state_dict()[k], trained_model['model'].state_dict()[k])
                  for k in yolo_model['model'].state_dict().keys())
print(f"Same weights: {same_weights}")


In [ ]:
import torch
from ultralytics import YOLO

# Load the working YOLO model
model = YOLO('runs/detect/firstRun/weights/best.pt')
yolo_model = YOLO('yolov9s.pt')

# Access the state dictionary directly from the model
state_dict = yolo_model.model

# Save this state_dict into a new file
torch.save({'model': state_dict}, 'runs/detect/firstRun/weights/best_with_weights.pt')

print("Weights saved successfully!")


In [ ]:
import yaml

# Load the class names from the data.yaml file
with open('pokemon_platinum_ai-14/data.yaml', 'r') as file:
    yaml_file = yaml.safe_load(file)
class_names = yaml_file['names']

# Create a dictionary mapping index to class name
names_order = list(range(len(class_names)))
class_dict = dict(zip(names_order, class_names))

print(class_dict)

model.model.names= class_dict
torch.save({'model': state_dict}, 'runs/detect/firstRun/weights/best_with_weights.pt')

In [ ]:
trained_model['model'].names

In [ ]:
model = torch.load('runs/detect/firstRun/weights/best_with_weights.pt')
print(model['model'])

model['model'].names = class_names

In [19]:
import requests
import dotenv
from roboflow import Roboflow
import os
import dill


dotenv.load_dotenv()
rf = Roboflow(api_key= os.getenv("ROBOFLOW_APIKEY"))
project = rf.workspace("scotts-workspace").project("pokemon_platinum_ai")
version = project.version(14)
version.deploy(model_type="yolov9s", model_path="runs/detect/firstRun", filename = "weights/best_with_weights.pt")


# if response.status_code == 200:
#     print("Model uploaded successfully!")
# else:
#     print(f"Failed to upload the model: {response.content}")


loading Roboflow workspace...
loading Roboflow project...
View the status of your deployment at: https://app.roboflow.com/scotts-workspace/pokemon_platinum_ai/15
Share your model with the world at: https://universe.roboflow.com/scotts-workspace/pokemon_platinum_ai/model/15
